# Neuron Classification Using Neural Networks in Julia

This project demonstrates how to classify neurons based on their firing patterns using a simple neural network implemented in Julia. It covers various aspects of data science, machine learning, and computational neuroscience.

## Step 1 - Generate Synthetic Data

Start by generating a synthetic dataset of neurons. Each neuron has a unique ID, a firing rate (how often it sends a signal), and a type (either excitatory or inhibitory).

In [ ]:
using Random, CSV, DataFrames

function generate_data(n::Int)
    data = DataFrame(id = 1:n, firing_rate = rand(50:150, n), class = rand(["Excitatory", "Inhibitory"], n))
    CSV.write("neuron_data.csv", data)
end

# Generate data
generate_data(1000)

## Step 2 - Load and Explore the Data

Next, the generated data loads from the CSV file and displays the first few rows to understand its structure. Then the data is explored by summarizing its statistics.

In [ ]:
using CSV, DataFrames

# Load the data and display the first 5 rows
data = CSV.read("neuron_data.csv", DataFrame)
println(first(data, 5))

# Explore the data
println(describe(data))

## Step 3 - Data Processing

The data needs to be processed to make it suitable for training a neural network. This involves converting the neuron types into a format that the neural network can understand and splitting the data into training and testing sets.

In [ ]:
using MLJ

# Encode categorical variables
data.class = coerce(data.class, Multiclass)

# Split the data
using MLJBase

train, test = partition(eachindex(data.id), 0.8, shuffle=true)
train_data = data[train, :]
test_data = data[test, :]

## Step 4 - Define the Neural Network Model

Define a simple neural network using the Flux library. This network will learn to classify neurons based on their firing rates.


In [ ]:
using Flux
using Optimisers  # Import optimisers for ADAM

model = Chain(
    Dense(1, 10, relu),
    Dense(10, 2),
    softmax
)

## Step 5 - Train the Neural Network

Prepare the training data and train the neural network. The network adjusts its internal settings to learn how to classify neurons.


In [ ]:
X_train = Matrix(train_data[:, :firing_rate]')
y_train = Flux.onehotbatch(train_data.class, ["Excitatory", "Inhibitory"])

# Define the loss function and optimizer
loss(model, x, y) = Flux.crossentropy(model(x), y)
opt = Flux.setup(Optimisers.ADAM(), model)

# Training loop
data = [(X_train, y_train)]
parameters = Flux.params(model)
for epoch in 1:100
    Flux.train!(loss, params(model), data, opt)
end

# Print the trained model
println("Trained Model:")
println(model)

## Step 6 - Evaluate the Neural Network

Evaluate the trained neural network using the test data. This involves making predictions and calculating the accuracy of the model.


In [ ]:
# Prepare data for testing
X_test = Matrix(test_data[:, :firing_rate]')
y_test = test_data.class

# Make predictions
y_pred = model(X_test)
y_pred_labels = Flux.onecold(y_pred, ["Excitatory", "Inhibitory"])

# Calculate accuracy
accuracy = mean(y_pred_labels .== y_test)
println("Model Accuracy: ", accuracy)

## Conclusion

This project demonstrates how to use Julia and Flux to build a neural network for classifying neurons based on their firing patterns. The final output is the accuracy of the model, which indicates how well the network can classify new neurons.
